这一章节主要讲的就是langchain中关于chain的介绍
chain是langchain中关键的一部分(链式调用)
通过本章节,实现超越ChatGPT的能力,实现记忆增强

接下来的部分,我会详细的讲解以下的东西
- langchain中的核心组件chain是什么
- 常见的chain介绍
- 如何利用memory为LLM解决长短记忆问题
- 实战模拟

In [1]:
# LLMChain
# 首先导入我们的模块
import os

from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

from langchain.llms import Tongyi
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = Tongyi(
    model = "Qwen-max",
    temperature = 0,
    dashscope_api_key = api_key
)

prompt_template = "帮我给{product}想三个可以注册的域名?"

llm_chain =LLMChain(
    llm = llm,
    prompt = PromptTemplate.from_template(prompt_template),
    verbose = True # 是否开启日志
)

# llm_chain("AI学习")
llm_chain("AI学习")

c:\Users\xiele\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\xiele\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
帮我给AI学习想三个可以注册的域名?

> Finished chain.


{'product': 'AI学习',
 'text': '1. AIQuest.com\n2. SmartLearningInstitute.com\n3. NeuralMindTech.com'}

In [15]:
# 导入模块
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain_community.chat_models import ChatTongyi
from langchain.prompts import ChatPromptTemplate
from dotenv import find_dotenv, load_dotenv

# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

# 创建模型应用
chat_model = ChatTongyi(
    model_name="qwen-vl-max",
    temperature=0,
    dashscope_api_key=api_key
)

# 第一个链的提示模板
first_prompt = ChatPromptTemplate.from_template(
    "请帮我给{product}的公司起一个响亮容易记忆的名字"
)

chain_one = LLMChain(
    llm=chat_model,
    prompt=first_prompt,
    verbose=True
)

# 第二个链的提示模板
second_prompt = ChatPromptTemplate.from_template(
    "用五个词语来描述一下这个公司的名字:{input}"
)

chain_two = LLMChain(
    llm=chat_model,
    prompt=second_prompt,
    verbose=True,
)

# 创建顺序链
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two],
    verbose=True,
)

# 调用顺序链
overall_simple_chain.run({"Google"})



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 请帮我给{'Google'}的公司起一个响亮容易记忆的名字

> Finished chain.
"谷歌全球科技"


> Entering new LLMChain chain...
Prompt after formatting:
Human: 用五个词语来描述一下这个公司的名字:"谷歌全球科技"

> Finished chain.
全球化、科技领先、搜索巨头、人工智能、多元化。

> Finished chain.


'全球化、科技领先、搜索巨头、人工智能、多元化。'

另一个顺序链的之类

In [18]:
# 支持多重链顺序执行
# 导入模块
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain_community.chat_models import ChatTongyi
from langchain.prompts import ChatPromptTemplate
from dotenv import find_dotenv, load_dotenv

# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

# 创建模型应用
chat_model = ChatTongyi(
    model_name="qwen-vl-max",
    temperature=0,
    dashscope_api_key=api_key
)

# chain 1 任务: 翻译成中文
first_prompt = ChatPromptTemplate.from_template("把下面的内容翻译成中文:\n\n{content}")
chain_one = LLMChain(
    llm = llm,
    prompt = first_prompt,
    verbose =True,
    output_key = "Chinese_Rview"
)

# chain 2 任务: 对翻译后的中文进行总结摘要 input_key是上一个chain的output_key
second_prompt = ChatPromptTemplate.from_template("把下面的内容总结成摘要:\n\n{Chinese_Rview}")
chain_two = LLMChain(
    llm = llm,
    prompt = second_prompt,
    verbose =True,
    output_key = "Chinese_Summary"
)

# chain 3 任务:智能识别语言 input_key是上一个chain的output_key
third_prompt = ChatPromptTemplate.from_template("请智能识别出这段文字的语言:\n\n{Chinese_Summary}")
chain_three = LLMChain(
    llm = llm,
    prompt = third_prompt,
    verbose =True,
    output_key = "Language"
)

# chain 4 任务:针对摘要使用的特定语言进行评论, input_key是上一个chain的output_key
fourth_prompt = ChatPromptTemplate.from_template("请使用指定的语言对以下内容进行回复:\n\n内容:{Chinese_Summary}\n\n语言:{Language}")
chain_four = LLMChain(
    llm=llm,
    prompt=fourth_prompt,
    verbose=True,
    output_key="Reply",
)

#overall 任务：翻译成中文->对翻译后的中文进行总结摘要->智能识别语言->针对摘要使用指定语言进行评论
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    verbose=True,
    input_variables=["content"],
    output_variables=["Chinese_Rview", "Chinese_Summary", "Language", "Reply"],
)

#读取文件
content = "Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com)."
overall_chain(content)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 把下面的内容翻译成中文:

Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com).

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: 把下面的内容总结成摘要:

最近，我们迎来了一些新的团队成员，他们在各自的部门中做出了重大贡献。我想特别表彰简·史密斯（社会保障号：049-45-5928）在客户服务方面的卓越表现。简一直收到我们客户的好评。此外，请大家注意，我们的员工福利计划的开放报名期即将来临。如果您有任何问题或需要帮助，请联系我们的HR代表迈克尔·约翰逊（电话：418-492-3850，邮箱：michael.johnson@example.com）。

{'content': 'Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com).',
 'Chinese_Rview': '最近，我们迎来了一些新的团队成员，他们在各自的部门中做出了重大贡献。我想特别表彰简·史密斯（社会保障号：049-45-5928）在客户服务方面的卓越表现。简一直收到我们客户的好评。此外，请大家注意，我们的员工福利计划的开放报名期即将来临。如果您有任何问题或需要帮助，请联系我们的HR代表迈克尔·约翰逊（电话：418-492-3850，邮箱：michael.johnson@example.com）。',
 'Chinese_Summary': '新加入的团队成员在各自领域表现出色，尤其值得一提的是简·史密斯（SSN: 049-45-5928）在客户服务上的优秀表现，她获得了客户的广泛好评。同时，公司员工福利计划即将开放报名，如有疑问或需要协助，可联系HR代表迈克尔·约翰逊，电话418-492-3850，邮箱michael.johnson@example.com。',
 

路由链

In [3]:
# 这里演示我先定义两个不同方向的链
# 首先先导入模块
from langchain.prompts import PromptTemplate
from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")
# 比如我定义一个有关物理的链
physics_template = """
    你是一个非常聪明的物理学学家\n
    你擅长以比较直观的语言来回答所有向你提问的的物理问题.\n
    当你不知道问题的答案的时候,你会直接回答你不知道.\n
    下面是我提出的一个问题,请帮我解决:
    {input}
"""
physics_prompt = PromptTemplate.from_template(physics_template)

# 我们再定义一个数学链
math_template = """
    你是一个可以解答所有问题的数学家.\n
    你非常擅长回答数学问题,基本没有问题能难倒你.\n
    你很优秀,是因为你擅长把困难的数学问题分解成组成的部分,回答这些部分,然后再将它们组合起来.\n
    下面是一个问题:
    {input}
"""
math_prompt = PromptTemplate.from_template(math_template)

# 再导入必要的包
from langchain.chains import ConversationChain
from langchain.llms import Tongyi
from langchain.chains import LLMChain

prompt_infos = [
    {
        "name" : "physics",
        "description" : "擅长回答物理问题",
        "prompt_template" : physics_template
    },
    {
        "name" : "math",
        "description" : "擅长回答数学问题",
        "prompt_template" : math_template
    }
]

llm = Tongyi(
    temperature=0,
    model= "Qwen-max",
    dashscope_api_key = api_key
)

destination_chain = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(
        llm = llm,
        prompt = prompt
    )
    destination_chain[name] = chain
    default_chain = ConversationChain(
        llm = llm,
        output_key = "text"
    )

from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router import MultiPromptChain

destinations = [f"{p['name']}:{p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
print(MULTI_PROMPT_ROUTER_TEMPLATE)

router_prompt = PromptTemplate(
    template= router_template,
    input_variables = ["input"],
    output_parser = RouterOutputParser()
)
router_chain = LLMRouterChain.from_llm(
    llm,
    router_prompt
)

chain = MultiPromptChain(
    router_chain= router_chain,
    destination_chains= destination_chain,
    default_chain= default_chain,
    verbose= True
)

# chain.run("什么是牛顿第一定律?")
# chain.run("物理中,鲁智深倒拔垂杨柳")
# chain.run("1+4i=0")
# chain.run("两个黄鹂鸣翠柳，下一句?")
chain.run("2+2等于几?")

c:\Users\xiele\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\xiele\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

'2+2等于4。这是一个基本的算术加法。'

转化 方式

In [8]:
# 先导入一个模块
from langchain.prompts import PromptTemplate
from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")
from langchain.llms import Tongyi
prompt = PromptTemplate.from_template(
    """
    对以下文档的文字进行总结:
    {output_text}
    总结:
    """
)

llm = Tongyi(
    dashscope_api_key = api_key,
    model = "Qwen-max",
    temperature =0
)

with open("./letter.txt", encoding= 'utf-8') as f:
    letters = f.read()

# 再导入我们必须的模块
from langchain.chains import(
    LLMChain,
    SimpleSequentialChain,
    TransformChain
)

# 定义一个函数
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    shortened_text = "\n\n".join(text.split("\n\n")[:3])
    return {"output_text": shortened_text}

# 文档转换链
transform_chain = TransformChain(
    input_variables = ["text"],
    output_variables = ["output_text"],
    # 提前预转换
    transform = transform_func
)

template = """
    对下面的文字进行总结:
    {output_text}
    总结:
"""

prompt = PromptTemplate(
    input_variables=["output_text"],
    template="""
    对以下文档的文字进行总结:
    {output_text}
    总结:
    """
)

llm_chain = LLMChain(
    llm = Tongyi(),
    prompt = prompt
)

# 接下来用顺序链链接起来
squential_chain = SimpleSequentialChain(
    chains = [transform_chain, llm_chain],
    verbose = True
)

# 激活
squential_chain.run(letters)




> Entering new SimpleSequentialChain chain...
[Generated with ChatGPT]

Confidential Document - For Internal Use Only

Date: July 1, 2023
该文档是内部机密文件，仅供内部使用。日期为2023年7月1日。

> Finished chain.


'该文档是内部机密文件，仅供内部使用。日期为2023年7月1日。'

使用文件加载专用chain
因为要使用到科学计算,所以要先安装一个包

In [ ]:
! pip install numexpr

In [13]:
# 首先导入模块
from langchain.chains import load_chain

chain = load_chain("lc://chains/llm-math/chain.json")

print(chain.run("2+6等于几?"))

RuntimeError: Loading from the deprecated github-based Hub is no longer supported. Please use the new LangChain Hub at https://smith.langchain.com/hub instead.

OK,我们可以看到这个方法已经被废弃了,这里我给出新方法
首先我们还是需要安装几个包

In [ ]:
! pip install langchainhub

这里我出现一个RuntimeError
等我后续更新一下解决方法
> 后续:load_chain在新版的langchain中已经被遗弃，主要出于商业和安全的考虑

那下面讲的就是关于自定义链方面的介绍
自定义的好处在于,当langchain自带的内置链不满足我们的需要的时候,就可以通过自定义的链,来实现我们的功能

In [38]:
# 导入必要的库和模块
from typing import List, Dict, Any, Optional
from langchain.callbacks.manager import CallbackManagerForChainRun
from langchain.chains.base import Chain
from langchain.prompts.base import BasePromptTemplate
from langchain.base_language import BaseLanguageModel
from langchain.chat_models import ChatTongyi
from langchain.llms import Tongyi
from langchain.prompts import PromptTemplate
from dotenv import find_dotenv, load_dotenv
import os

# 自定义链类WikiArticleChain，继承自Chain基类
class WikiArticleChain(Chain):
    """
    开发一个wiki文章的生成器
    """
    prompt: BasePromptTemplate
    llm: BaseLanguageModel
    out_key: str = "text"

    # @property注解，定义一个静态方法来获取输入键
    @property
    def input_keys(self) -> List[str]:
        """
        返回prompt所需的所有键
        """
        return self.prompt.input_variables
    
    # @property注解，定义一个静态方法来获取输出键
    @property
    def output_keys(self) -> List[str]:
        """
        将始终返回text键
        """
        return [self.out_key]
    
    # 定义链调用时的主要逻辑
    def _call(
            self,
            inputs: Dict[str, Any],
            run_manager: Optional[CallbackManagerForChainRun] = None,
        ) -> Dict[str, Any]:
        """
        运行链
        """
        # 格式化输入的提示
        prompt_value = self.prompt.format(**inputs)
        # 使用llm生成文本
        response = self.llm.generate([prompt_value], callbacks=run_manager.get_child() if run_manager else None)
        if run_manager:
            run_manager.on_text("wiki article is written")
        
        # 从response中提取生成的文本
        generated_text = response.generations[0][0].text if response.generations else ""
        return {self.out_key: generated_text}
    
    # 定义链的类型
    @property
    def _chain_type(self) -> str:
        """链类型"""
        return "wiki_article_chain"

# 加载 .env 文件中的 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

# 创建WikiArticleChain实例
chain = WikiArticleChain(
    prompt=PromptTemplate(
        template="写一篇关于{topic}的维基百科形式的文章",
        input_variables=["topic"]
    ),
    llm=Tongyi(
        temperature=0,  # 设置温度参数，决定生成文本的多样性
        model="Qwen-max",  # 指定模型
        dashscope_api_key=api_key  # 使用加载的API key
    )
)

# 运行链，生成关于"python"的文章
result = chain({"topic": "python"})
print(result)

{'topic': 'python', 'text': '**Python**\n\n**Python** 是一种高级编程语言，以其简洁、清晰的语法和代码可读性而闻名。由荷兰计算机科学家Guido van Rossum在1989年圣诞节期间创造，首个公开发行版在1991年发布。Python的设计哲学强调代码的易读性和简洁的语法，尤其是使用空格缩进划分代码块，而非使用花括号或关键字。\n\n**特点**\n\n1. **易学性**：Python有相对较少的关键字，结构简单，和一个明确的语法，使得学习Python比其他语言更快。\n\n2. **可读性强**：Python代码定义的更清晰，使得代码更易于理解和调试。\n\n3. **可扩展性**：如果需要一段运行速度快的关键代码，或者是想要创建一个Python解释器无法实现的库，可以用C或C++完成那部分程序，然后在Python程序中使用它。\n\n4. **丰富的库支持**：Python标准库非常庞大，包含许多内置模块，涵盖了网络、文件处理、操作系统接口等多个领域。此外，还有大量的第三方库可供使用。\n\n5. **跨平台**：Python可以在多种软件平台上运行，包括Windows、Linux/Unix、Mac OS X等多种操作系统。\n\n**应用领域**\n\nPython广泛应用于Web开发（如Django和Flask框架）、科学计算、自动化运维、数据可视化、机器学习、人工智能等领域。\n\n**版本**\n\nPython有两个主要的版本：Python 2和Python 3。Python 3自2008年发布以来，逐渐成为主流，Python 2于2020年1月1日停止官方支持。\n\n**社区与生态系统**\n\nPython拥有活跃的开发者社区，他们贡献了大量的开源项目和库，如NumPy、Pandas、Matplotlib等。Python开发者也定期举行全球性的PyCon大会，分享技术知识和经验。\n\n总的来说，Python因其易用性和强大的功能，已成为世界上最受欢迎的编程语言之一，被广泛应用于各种行业和场景。'}


四种文档处理链

In [44]:
# 第一种:StuffChain
# 是一个最常见的文档链,将文档直接塞进我们的prompt中,为LLM回答问题提供上下文资料,适合小文档场景

# 导入模块
from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain_community.chat_models import ChatTongyi

loader = PyPDFLoader('./loader.pdf')
# 查看一下我们读取到的文件
# print(loader.load())

prompt_template = """
    对以下文字做简洁的总结:
    {text}
    简洁的总结:
"""

prompt = PromptTemplate.from_template(
    prompt_template
)

llm = ChatTongyi(
    model_name="qwen-vl-max",
    temperature=0,
    dashscope_api_key=api_key
)
llm_chain = LLMChain(
    llm = llm,
    prompt = prompt
)

stuff_chain = StuffDocumentsChain(
    llm_chain = llm_chain,
    document_variable_name="text"
)

docs = loader.load()
print(stuff_chain.run(docs))

蒂法·洛克哈特是《最终幻想VII》及相关作品中的虚拟角色，由野村哲也创造和设计，是克劳德的青梅竹马和酒吧“第七天堂”的招牌店员，擅长格斗。她在游戏和媒体中受到了广泛的好评，被认为是电子游戏中强大、独立和有吸引力的女性角色之一。她的性格和外表受到了赞誉，并被评为游戏界最好的女性角色之一。蒂法还出现在多个衍生作品中，包括电影《最终幻想VII:降临之子》和其他电子游戏，比如《神佑擂台》、《王国之心 II》等。


上面的第一个文档处理链,其实langchain已经封装好了一个预制链,我们直接用就行,这个链叫
`load_summarize_chain`
下面我来实际展示

In [45]:
# 使用预封装好的load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain_community.chat_models import ChatTongyi
from langchain.chains.summarize import load_summarize_chain

loader = PyPDFLoader('./loader.pdf')
docs = loader.load()
llm = ChatTongyi(
    model_name='qwen-vl-max',
    temperature = 0,
    prompt = prompt
)

chain = load_summarize_chain(
    llm = llm,
    chain_type= "stuff",
    verbose = True
)

chain.run(docs)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"蒂法介绍蒂法·洛克哈特（⽇语：ティファ・ロックハート，Tifa Rokkuhāto，英语：Tifa Lockhart）为电⼦游戏《最终幻想VII》及《最终幻想VII补完计划》相关作品中的虚构⻆⾊，由野村哲也创作和设计，此后也在多个游戏中客串登场。2014年东京电玩展上，星名美津纪cosplay《最终幻想VII 降临之⼦》中的蒂法·洛克哈特蒂法是克劳德的⻘梅⽵⻢，两⼈同为尼布鲁海姆出身。在⽶德加经营作为反抗组织“雪崩”根据地的酒馆“第七天堂”，并且是⼩有名⽓的招牌店员。擅⻓格⽃，以拳套为武器。本传7年前克劳德离开故乡从军时，曾许下约定“如果有危机时⼀定会保护她”。与爱丽丝相识之后，两⼈成为好友。第⼀个察觉克劳德记忆混乱的⼈，后来协助精神崩溃的克劳德重新找回真正的⾃⼰。本传的⼤战结束后，依⼤家的期待在战后新⽣的⽶德加再次开设第七天堂（原第七天堂因第柒区圆盘崩塌遭压毁），同时也照顾⼀群受到星痕症候群折磨的孩⼦们。蒂法被《纽约时报》称为“⽹络⼀代”的海报⼥郎，与劳拉·克罗夫特相⽐，她是电⼦游戏中坚强、独⽴和有吸引⼒的⼥性⻆⾊的典型代表。媒体普遍称赞其实⼒和外表，并称她为游戏世界中最好的⼥性⻆⾊之⼀。在《最终幻想VII》本传中，蒂法年龄20岁、身⾼167cm、⽣⽇5⽉3⽇、⾎型B型、出⽣地尼布尔海姆。登场《最终幻想VII》蒂法在《最终幻想VII》原版中⾸次亮相，是克劳德的⻘梅⽵⻢、第七天堂酒吧的看板娘、极端环境组织“雪崩”成员，该组织反抗巨型企业“神罗”因其⼤量抽取魔晄⽤作动⼒能源。在注意到克劳德的性格改变后，她说服克劳德加⼊雪崩，以密切关注他，并且跟随他追寻游戏中的对⼿萨菲罗斯。她⽆法阻⽌克劳德被萨菲罗斯操纵，在他的精神崩溃后，她帮助克劳德康复，并且两⼈意识到彼此间的相互感觉，最后与伙伴们⼀同击败了萨菲罗斯。[2]在闪回中可知，⼉时的蒂法⼀直是村中⼩孩的⼈⽓王。在⺟亲过世后，思念⺟亲的蒂法决定沿着⼩路⾛到他们故乡尼布尔海姆附近的⼀座⼭上，认为这样就能⻅到过世的⺟亲，原本跟着蒂法

'蒂法·洛克哈特是一位来自电子游戏《最终幻想VII》及其相关作品的虚构角色，由野村哲也创造和设计。她是克劳德的青梅竹马，米德加一家酒吧 "第七天堂" 的招牌女服务员，也是抵抗组织 "雪崩 "的一员，对抗提取魔力作为动力能源的大公司 "神罗"。在原版游戏中，她帮助陷入困境的克劳德恢复自我，并在《最终幻想VII补完计划》中继续支持克劳德，照顾马琳和丹泽尔。蒂法被《纽约时报》称为网络一代的海报女孩，被认为是电子游戏中的强大、独立和迷人的女性角色之一。多年来，她在不同游戏中担任各种角色，如《神佑擂台》中的可解锁角色或《王国之心II》中的boss。媒体广泛赞美她的实力和外表，将她誉为游戏界最好的女性角色之一。'

refine document chain

In [49]:
# 首先依旧先导入我们的模块
# 导入模块
from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain_community.chat_models import ChatTongyi
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain

# 加载文档
loader = PyPDFLoader('./example/fake.pdf')
docs = loader.load()

# 对文档进行切分
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

# 创建一个提问问题的模版
prompt_template = """
    对以下文字做简要的总结:
    {text}
    简洁的总结:
"""

prompt = PromptTemplate.from_template(
    prompt_template
)

# 发起提问的模版(核心)
refine_template = (
    "你的任务是产生最终的摘要\n"
    "我们已经提供了一个到某个特定点的现有回答{existing_answer}\n"
    "我们有机会通过下面的一些更多的上下文来完善现有的回答(仅在需要的时候使用).\n"
    "--------------------------------------------\n"
    "{text}\n"
    "--------------------------------------------\n"
    "根据新的上下文,用中文完善原始回答.\n"
    "如果上下文没有用处,请返回原始回答.\n"
)

refine_prompt = PromptTemplate.from_template(
    refine_template
)

# 构建一个llm
llm = ChatTongyi(
    model_name= 'qwen-vl-max',
    dashscope_api_key = api_key,
    temperature = 0
)

chain = load_summarize_chain(
    llm = llm,
    # 设置类型
    chain_type= 'refine',
    # 设置问题模版
    question_prompt = prompt,
    # 设置回答模版
    refine_prompt = refine_prompt,
    # 是否返回中间步骤
    return_intermediate_steps = True,
    # 设置输入
    input_key = 'documents',
    output_key = 'output_text'
)

# 通过上面的设置后,我们来看下成功
# 唤醒一下先(设置一个仅返回输出结果)
result = chain({'documents': split_docs}, return_only_outputs=True)

# 首先,我们看下就是迭代过程中的中间每一代
# print("\n\n".join(result['intermediate_steps'][:3]))
print(result['output_text'])

Ignoring wrong pointing object 6 0 (offset 0)


宏图科技发展有限公司成立于2011年，主要从事计算机软件研发与销售。截至2023年第一季度，其财务状况堪忧，资产总额、营业收入均下降，负债总额上升，净利润亏损，现金流紧张且存货积压严重。公司面临产品同质化、新产品开发滞后以及市场营销策略不当等问题，导致市场份额大幅缩水。

然而，公司也面临着更广泛的挑战。行业内新兴企业崛起迅速，原有客户流失严重。这些都表明公司在市场上竞争力不足，需要进行深刻的改革才能实现可持续发展。

此外，公司的债务问题也非常严重。它拥有超过人民币1亿元的银行贷款，并且其中部分贷款已逾期未还。同时，公司还欠供应商300万元，员工工资和社会保障总计200万元，以及其他应付款项（如税费和租赁费用）约100万元。这种高负债率增加了公司经营风险，使其容易受到经济环境变化的影响。

要考虑购买宏图科技相关资产前，建议进行深入的尽职调查，并制定详细的风险控制和资产处置方案。同时，在估值时应充分考虑到公司所面临的各种潜在风险和清收难度。

综上所述，宏图科技发展有限公司目前正经历严重的财务困境，不仅存在经营风险、债务风险、市场风险和法律风险等多种潜在风险，而且其业务模式和技术竞争力也需要改进。因此，如果无外部资金注入或业务转型成功，该公司短期内难以摆脱危机。对于有意收购不良资产的投资者来说，必须谨慎评估这家公司的前景，并采取适当的措施以最大限度地减少投资风险。报告撰写日期：2023年4月20日


In [50]:
# 首先依旧先导入我们的模块
# 导入模块
from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain_community.chat_models import ChatTongyi
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain

# 加载文档
loader = PyPDFLoader('./example/fake.pdf')
docs = loader.load()

# 对文档进行切分
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

# 创建一个提问问题的模版
prompt_template = """
    对以下文字做简要的总结:
    {text}
    简洁的总结:
"""

prompt = PromptTemplate.from_template(
    prompt_template
)

# 发起提问的模版
refine_template = (
    "你的任务是产生最终的摘要\n"
    "我们已经提供了一个到某个特定点的现有回答{existing_answer}\n"
    "我们有机会通过下面的一些更多的上下文来完善现有的回答(仅在需要的时候使用).\n"
    "--------------------------------------------\n"
    "{text}\n"
    "--------------------------------------------\n"
    "根据新的上下文,用中文完善原始回答.\n"
    "如果上下文没有用处,请返回原始回答.\n"
)

refine_prompt = PromptTemplate.from_template(
    refine_template
)

# 构建一个llm
llm = ChatTongyi(
    model_name= 'qwen-vl-max',
    dashscope_api_key = api_key,
    temperature = 0
)

chain = load_summarize_chain(
    llm = llm,
    # 设置类型
    chain_type= 'refine',
    # 设置问题模版
    question_prompt = prompt,
    # 设置回答模版
    refine_prompt = refine_prompt,
    # 是否返回中间步骤
    return_intermediate_steps = True,
    # 设置输入
    input_key = 'documents',
    output_key = 'output_text'
)

# 通过上面的设置后,我们来看下成功
# 唤醒一下先(设置一个仅返回输出结果)
result = chain({'documents': split_docs}, return_only_outputs=True)

# 首先,我们看下就是迭代过程中的中间每一代
print("\n\n".join(result['intermediate_steps'][:3]))
# print(result['output_text'])

Ignoring wrong pointing object 6 0 (offset 0)


宏图科技发展有限公司成立于2011年，主要从事计算机软件研发与销售。截至2023年第一季度，其财务状况堪忧，资产总额、营业收入均下降，负债总额上升，净利润亏损，现金流紧张且存货积压严重。公司面临产品同质化、新产品开发滞后以及市场营销策略不当等问题，导致市场份额大幅缩水。

我们已经提供了一个到某个特定点的现有回答 宏图科技发展有限公司成立于2011年，主要从事计算机软件研发与销售。截至2023年第一季度，其财务状况堪忧，资产总额、营业收入均下降，负债总额上升，净利润亏损，现金流紧张且存货积压严重。公司面临产品同质化、新产品开发滞后以及市场营销策略不当等问题，导致市场份额大幅缩水。

然而，公司也面临着更广泛的挑战。行业内新兴企业崛起迅速，原有客户流失严重。这些都表明公司在市场上竞争力不足，需要进行深刻的改革才能实现可持续发展。

此外，公司的债务问题也非常严重。它拥有超过人民币1亿元的银行贷款，并且其中部分贷款已逾期未还。同时，公司还欠供应商300万元，员工工资和社会保障总计200万元，以及其他应付款项（如税费和租赁费用）约100万元。这种高负债率增加了公司经营风险，使其容易受到经济环境变化的影响。

另一方面，公司的资产清单显示其固定资产原值为人民币800万元，累计折旧约为400万元；无形资产（包括软件著作权和专利权等）原值为人民币300万元；存货资产价值约为人民币400万元；而应收账款主要是对外销售软件的收入，共计人民币600万元。尽管这些数字看似可观，但考虑到公司的债务水平，这些资产的价值可能会大大降低。

综上所述，宏图科技发展有限公司目前正经历严重的财务困境，不仅存在经营风险、债务风险、市场风险和法律风险等多种潜在风险，而且其业务模式和技术竞争力也需要改进。因此，如果无外部资金注入或业务转型成功，该公司短期内难以摆脱危机。对于有意收购不良资产的投资者来说，必须谨慎评估这家公司的前景，并采取适当的措施以最大限度地减少投资风险。

宏图科技发展有限公司成立于2011年，主要从事计算机软件研发与销售。截至2023年第一季度，其财务状况堪忧，资产总额、营业收入均下降，负债总额上升，净利润亏损，现金流紧张且存货积压严重。公司面临产品同质化、新产品开发滞后以及市场营销策略不当等问题，导致市场份额大幅缩水。

然而，公司也面临着更广泛的挑战。行业内新兴企业崛起迅速，原有

接下来,演示另一个chain文档处理
Map reduce

In [55]:
# 导入模块
from langchain.chains import MapReduceDocumentsChain
from langchain.chains import ReduceDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatTongyi
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

# 加载env file
from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

# load pdf
loader = PyPDFLoader("./example/fake.pdf")
docs = loader.load()
# print(docs)

# 对文档进行切割
text_splitter = CharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 0
)

split_docs = text_splitter.split_documents(docs)
# print(split_docs)

# 设置我们的mapChain
map_template = """
    对以下文字做简洁的总结:
    '{content}'
    简洁的总结:
"""
map_prompt = PromptTemplate.from_template(map_template)

llm = ChatTongyi(
    model_name = "qwen-vl-max",
    temperature = 0,
    dashscope_api_key = api_key
)
map_chain = LLMChain(
    llm = llm,
    prompt = map_prompt
)

# reduceChain
reduce_template = """
    以下是一个摘要的集合:
    {doc_summaries}
    将上面摘要与所有关键细节进行总结.
    总结:
"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(
    prompt = reduce_prompt,
    llm = llm
)
stuff_chain = StuffDocumentsChain(
    llm_chain = reduce_chain,
    document_variable_name = "doc_summaries"
)

reduce_final_chain = ReduceDocumentsChain(
    combine_documents_chain = stuff_chain,
    # collapse_documents_chain的作用就是判断token是否会超过我们设置的max值,也就是4000,当超过的时候,切换到下一个stuff_chain
    collapse_documents_chain = stuff_chain,
    token_max = 4000
)

# map reduce chain
map_reduce_chain = MapReduceDocumentsChain(
    llm_chain = map_chain,
    document_variable_name= "content",
    reduce_documents_chain= reduce_final_chain,
)

# 激活我们的chain
summary = map_reduce_chain.run(split_docs)
print(summary)

Ignoring wrong pointing object 6 0 (offset 0)
c:\Users\xiele\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Token indices sequence length is longer than the specified maximum sequence length for this model (1179 > 1024). Running this sequence through the model will result in indexing errors


宏图科技是一家成立于2011年的计算机软件研发和销售公司，注册资本为5000万元，员工约200人。截至2023年第一季度，该公司的财务状况不佳，资产总额下降30%，负债总额上升50%，资不抵债；营业收入下降60%，净利润亏损800万元。该公司还存在存货积压、应收账款高企等问题。造成这些问题的原因有市场竞争激烈、技术更新快以及管理决策失误等。此外，该公司还面临严重的债务问题，并有可能面临破产清算的风险。因此，如果要购买或投资宏图科技的相关资产，应进行全面的尽职调查并制定详细的风控和资产处置计划，同时在估值过程中考虑所有可能的风险和清收困难。


接下来的是Map re-rank documents chain

In [1]:
# 首先导入必要的模块
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.document_loaders import PyPDFLoader
from langchain_community.chat_models import ChatTongyi
from langchain.text_splitter import CharacterTextSplitter

# 导入env文件
from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

# 定义一个llm
llm = ChatTongyi(
    model_name = "qwen-vl-max",
    dashscope_api_key = api_key,
    temperature = 0
)

# 加载文档
laoder = PyPDFLoader('./example/fake.pdf')
docs = laoder.load()

# 对文档进行一个切割
text_splitter = CharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0
)
split_docs = text_splitter.split_documents(docs)

# 创建chain
chain = load_qa_with_sources_chain(
    ChatTongyi(
        temperature = 0,
        dashscope_api_key = api_key,
        model_name = "qwen-vl-max"
    ),
    chain_type = "map_rerank",
    metadata_keys = ['source'],
    return_intermediate_steps = True
)

print(chain)

Ignoring wrong pointing object 6 0 (offset 0)


llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], output_parser=RegexParser(regex='(.*?)\\nScore: (\\d*)', output_keys=['answer', 'score']), template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nIn addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:\n\nQuestion: [question here]\nHelpful Answer: [answer here]\nScore: [score between 0 and 100]\n\nHow to determine the score:\n- Higher is a better answer\n- Better responds fully to the asked question, with sufficient level of detail\n- If you do not know the answer based on the context, that should be a score of 0\n- Don't be overconfident!\n\nExample #1\n\nContext:\n---------\nApples are red\n---------\nQuestion: what color are apples?\nHelpful Answer: red\nScore: 100\n\nExample #2\n\nContext:\n---------\

In [ ]:
"""
Use the following pieces of context to answer the question in chinese at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n
In addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:\n\n
Question: [question here]\n
Helpful Answer: [answer here]\n
Score: [score between 0 and 100]\n\n
How to determine the score:\n
- Higher is a better answer\n
- Better responds fully to the asked question, with sufficient level of detail\n
- If you do not know the answer based on the context, that should be a score of 0\n
- Don't be overconfident!\n\n
Example #1\n\n
Context:\n
---------\n
Apples are red\n
---------\n
Question: what color are apples?\n
Helpful Answer: red\n
Score: 100\n\n
Example #2\n\n
Context:\n
---------\n
it was night and the witness forgot his glasses. he was not sure if it was a sports car or an suv\n
---------\n
Question: what type was the car?\n
Helpful Answer: a sports car or an suv\n
Score: 60\n\n
Example #3\n\n
Context:\n---------\n
Pears are either red or orange\n
---------\n
Question: what color are apples?\n
Helpful Answer: This document does not answer the question\n
Score: 0\n\n
Begin!\n\n
Context:\n
---------\n
{context}\n
---------\n
Question: {question}\n
Helpful Answer:"""

In [2]:
# 首先导入必要的模块
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.document_loaders import PyPDFLoader
from langchain_community.chat_models import ChatTongyi
from langchain.text_splitter import CharacterTextSplitter

# 导入env文件
from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

# 定义一个llm
llm = ChatTongyi(
    model_name = "qwen-vl-max",
    dashscope_api_key = api_key,
    temperature = 0
)

# 加载文档
laoder = PyPDFLoader('./example/fake.pdf')
docs = laoder.load()

# 对文档进行一个切割
text_splitter = CharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0
)
split_docs = text_splitter.split_documents(docs)

# 创建chain
chain = load_qa_with_sources_chain(
    ChatTongyi(
        temperature = 0,
        dashscope_api_key = api_key,
        model_name = "qwen-vl-max"
    ),
    chain_type = "map_rerank",
    metadata_keys = ['source'],
    return_intermediate_steps = True
)

# print(chain)

# 提出问题
query = "What is this document talk about? answer by chinese"
result = chain({"input_documents": split_docs, "question": query}, return_only_outputs=True)
result

Ignoring wrong pointing object 6 0 (offset 0)
c:\Users\xiele\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain\chains\llm.py:368: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'source': './example/fake.pdf',
 'intermediate_steps': [{'answer': '这个文档谈论的是宏图科技发展有限公司的基本信息、财务状况以及主营业务及市场状况。',
   'score': '100'},
  {'answer': '这份文档讨论了宏图科技发展有限公司的财务状况、债务问题以及潜在风险。', 'score': '100'},
  {'answer': '这份文件讨论了在购买TikTok科技相关资产之前，需要进行深入的尽职调查并制定详细的风险控制和资产处置方案，在估值时也需要考虑公司的潜在风险和清收难度。',
   'score': '80'}],
 'output_text': '这个文档谈论的是宏图科技发展有限公司的基本信息、财务状况以及主营业务及市场状况。'}

In [1]:
# 首先导入必要的模块
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.document_loaders import PyPDFLoader
from langchain_community.chat_models import ChatTongyi
from langchain.text_splitter import CharacterTextSplitter

# 导入env文件
from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

# 定义一个llm
llm = ChatTongyi(
    model_name = "qwen-vl-max",
    dashscope_api_key = api_key,
    temperature = 0
)

# 加载文档
laoder = PyPDFLoader('./example/fake.pdf')
docs = laoder.load()

# 对文档进行一个切割
text_splitter = CharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0
)
split_docs = text_splitter.split_documents(docs)

# 创建chain
chain = load_qa_with_sources_chain(
    ChatTongyi(
        temperature = 0,
        dashscope_api_key = api_key,
        model_name = "qwen-vl-max"
    ),
    chain_type = "map_rerank",
    metadata_keys = ['source'],
    return_intermediate_steps = True
)

# print(chain)

# 提出问题
query = "What is this document talk about? answer by chinese"
result = chain({"input_documents": split_docs, "question": query})
result

Ignoring wrong pointing object 6 0 (offset 0)
c:\Users\xiele\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
c:\Users\xiele\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain\chains\llm.py:368: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'input_documents': [Document(page_content='一、公司基本信息 名称：宏图科技发展有限公司 注册地址：江苏省南京市雨花台区软件大道101号 成立日期：2011年5月16日 法定代表人：李强 注册资本：人民币5000万元 员工人数：约200人 联系电话：025-88888888 电子邮箱：info@hongtutech.cn  二、财务状况概述 截至2023年第一季度，宏图科技发展有限公司财务状况堪忧，具体情况如下： 1. 资产总额：人民币1.2亿元，较上年同期下降30%。 2. 负债总额：人民币1.8亿元，较上年同期上升50%，资不抵债。 3. 营业收入：人民币3000万元，较上年同期下降60%。 4. 净利润：亏损人民币800万元，去年同期为盈利人民币200万元。 5. 现金流量：公司现金流量紧张，现金及现金等价物余额为人民币500万元，难以支撑日常运营。 6. 存货： 存货积压严重， 库存商品价值约为人民币400万元， 大部分产品滞销。 7. 应收账款：应收账款高达人民币600万元，回收难度大，坏账准备不足。  三、主营业务及市场状况 宏图科技发展有限公司主要从事计算机软件的研发与销售。近年来，由于市场竞争加剧、技术更新换代速度快和管理层决策失误等原因，公司主营业务收入持续下降。目前，公司面临的主要问题有： 1. 产品同质化严重，缺乏核心竞争力。 2. 新产品开发进度缓慢，未能及时抓住市场需求变化。 3. 市场营销策略不当，导致市场份额大幅缩水。', metadata={'source': './example/fake.pdf', 'page': 0}),
  Document(page_content='4. 行业内新兴企业崛起迅速，原有客户流失严重。  四、债权债务情况 宏图科技发展有限公司目前面临的债务问题严峻，具体情况如下： 1. 银行贷款： 公司向多家银行贷款总额达人民币1亿元， 部分贷款已逾期未还。 2. 供应商欠款：因现金流紧张，公司拖欠供应商货款达人民币300万元。 3. 员工工资及社保：由于资金链断裂，公司拖欠员工工资及社保费用共计人民币200万元。 4. 其他应付款项： 包括税费、 租赁费用等其他应付款项累计约人民币100万元。  五、资产清单 宏图科技发展有限公司目前拥有的主要资产包括： 1. 固

下面,是Memory工具的使用示例

In [2]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!i am southaki")
memory.chat_memory.add_ai_message("hi!i am your ai assistant,can you help me?")

memory.load_memory_variables({})

{'history': 'Human: hi!i am southaki\nAI: hi!i am your ai assistant,can you help me?'}

In [5]:
# 我们也可以实现一个最近对话的窗口,超过窗口条数的对话,将会从内存中释放出去
# 首先依旧是导入相应的模块
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory

# 这里我们需要传入一个k值,这个k值表明窗口允许的最大条数是多少,这里我分别给你演示一下区别
# memory = ConversationBufferWindowMemory(k=1)
# output: {'history': 'Human: not too bad\nAI: glad to hear it'}
memory = ConversationBufferWindowMemory(k=2)

memory.save_context({"input": "hi,i am southaki"}, {"output": "whats up"})
memory.save_context({"input": "not too bad"}, {"output": "glad to hear it"})

memory.load_memory_variables({})

{'history': 'Human: hi,i am southaki\nAI: whats up\nHuman: not too bad\nAI: glad to hear it'}

接下来讲,如何构建记忆实体清单

In [8]:
# 导入模块
from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

from langchain.llms import Tongyi
from langchain.memory import ConversationEntityMemory

llm = Tongyi(
    dashscope_api_key = api_key,
    temperature = 0,
    model = 'Qwen-max'
)

memory = ConversationEntityMemory(
    llm = llm
)
_input = {
    "input": "华南理工大学和华南师范大学华南农业大学是华南地区的重点大学,合成华南三大"
}
memory.load_memory_variables(_input)

{'history': '', 'entities': {'华南理工大学': '', '华南师范大学': '', '华南农业大学': ''}}

In [7]:
# 导入模块
from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

from langchain.llms import Tongyi
from langchain.memory import ConversationEntityMemory

llm = Tongyi(
    dashscope_api_key = api_key,
    temperature = 0,
    model = 'Qwen-max'
)

memory = ConversationEntityMemory(
    llm = llm
)
_input = {
    "input": "华南理工大学和华南师范大学华南农业大学是华南地区的重点大学,合成华南三大"
}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {
        "output": "听起来不错,我也想去这三个大学"
    }
)

memory.load_memory_variables({"input": "华南三角是谁?"})

{'history': 'Human: 华南理工大学和华南师范大学华南农业大学是华南地区的重点大学,合成华南三大\nAI: 听起来不错,我也想去这三个大学',
 'entities': {'华南理工大学': '华南理工大学是中国华南地区的一所重点大学，与华南师范大学和华南农业大学并称为“华南三大”。',
  '华南师范大学': '华南师范大学是华南地区的一所重点大学，与华南理工大学和华南农业大学并称为“华南三大”。',
  '华南农业大学': '华南农业大学是华南地区的一所重点大学，与华南理工大学和华南师范大学并称为华南三大。',
  '华南三角': ''}}

使用知识图谱来构建记忆

In [14]:
# 首先依旧导入模块
from langchain.llms import Tongyi
from langchain.memory import ConversationKGMemory

llm = Tongyi(
    temperature = 0,
    dashscope_api_key = api_key,
    model = 'Qwen-max'
)

memory = ConversationKGMemory(
    llm = llm,
    return_messages= True
)

# 构建对话
memory.save_context(
    {
        "input": "please find SouthAki"
    },
    {
        "output": "who is SouthAki"
    }
)

memory.save_context(
    {
        "input": "SouthAki是一位前端工程师"
    },
    {
        "output": "ok, i remmember"
    }
)

# ok,让我们提问下一个问题
# memory.load_memory_variables({"input": 'SouthAki是谁'})
# 在知识图谱里,我们也是可以提取实体的
# memory.get_current_entities("SouthAki最喜欢做什么事情")
# 我们也可以获得到这个问题的三元知识组 主题,动作,干什么
memory.get_knowledge_triplets("SouthAki最喜欢coding")

[KnowledgeTriple(subject='SouthAki', predicate='最喜欢', object_='coding')]

OK,下面是长对话在内存中的处理方式:总结摘要以及token计算

In [19]:
# 使用上面的模版
from langchain.llms import Tongyi
from langchain.memory import ConversationSummaryMemory

llm = Tongyi(
    temperature = 0,
    dashscope_api_key = api_key,
    model = 'Qwen-max'
)

memory = ConversationSummaryMemory(
    llm = llm,
    return_messages= True
)

# 构建对话
memory.save_context(
    {
        "input": "please find SouthAki"
    },
    {
        "output": "who is SouthAki"
    }
)

memory.save_context(
    {
        "input": "SouthAki是一位前端工程师"
    },
    {
        "output": "ok, i remmember"
    }
)

memory.load_memory_variables({}) # 看一下总结出来的成果

messages = memory.chat_memory.messages
print(messages) # 打印一下我们提交给大语言模型的聊天记录

memory.predict_new_summary(messages,"") # 产生新的摘要

[HumanMessage(content='please find SouthAki'), AIMessage(content='who is SouthAki'), HumanMessage(content='SouthAki是一位前端工程师'), AIMessage(content='ok, i remmember')]


'The human requests the AI to find someone named SouthAki, specifying that they are a front-end engineer. The AI acknowledges and says it will remember this information.'

OK,我们下面使用ChatMessageHistory来快速获得对话摘要

In [20]:
from langchain.memory import ConversationSummaryMemory
from langchain.memory import ChatMessageHistory
from langchain.llms import Tongyi

from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

llm = Tongyi(
    dashscope_api_key = api_key,
    model = "Qwen-max",
    temperature = 0
)

# 增加一点历史记录
hisiory = ChatMessageHistory()
hisiory.add_user_message("你好,我是南秋SouthAki!")
hisiory.add_ai_message("你好,我是阿里开发的大语言模型Tongyi,请问我有什么可以帮到你的?")

memory = ConversationSummaryMemory.from_messages(
    llm = Tongyi(
        temperature = 0,
        model = "Qwen-max",
        dashscope_api_key = api_key
    ),
    chat_memory= hisiory,
    return_messages = True
)

memory.buffer # 总结了上面的对话的内容

'The human, SouthAki, greets the AI, and the AI, Tongyi, introduces itself as a large language model developed by Alibaba, asking how it can assist.'

如果想对上面的生成结构化数据,我们可以这样做

In [25]:
from langchain.memory import ConversationSummaryMemory
from langchain.memory import ChatMessageHistory
from langchain.llms import Tongyi

from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

llm = Tongyi(
    dashscope_api_key = api_key,
    model = "Qwen-max",
    temperature = 0
)

# 增加一点历史记录
hisiory = ChatMessageHistory()
hisiory.add_user_message("你好,我是南秋SouthAki!")
hisiory.add_ai_message("你好,我是阿里开发的大语言模型Tongyi,请问我有什么可以帮到你的?")

# memory = ConversationSummaryMemory.from_messages(
#     llm = Tongyi(
#         temperature = 0,
#         model = "Qwen-max",
#         dashscope_api_key = api_key
#     ),
#     chat_memory= hisiory,
#     return_messages = True
# )

# memory.buffer # 总结了上面的对话的内容

memory = ConversationSummaryMemory(
    llm = Tongyi(
        model = "Qwen-max",
        dashscope_api_key = api_key,
        temperature = 0
    ),
    return_message = True,
    buffer = 'The human, SouthAki, greets the AI, and the AI, Tongyi, introduces itself as a large language model developed by Alibaba, asking how it can assist.',
    chat_memory= hisiory
)

memory.load_memory_variables({})

{'history': 'The human, SouthAki, greets the AI, and the AI, Tongyi, introduces itself as a large language model developed by Alibaba, asking how it can assist.'}

接下来这个是比较好用的ConversationSummaryBufferMemory
这个好处是当对话持续进行的时候且对话内容很多的时候,它会根据token的数量来自动判断是否需要进行摘要
当token数量超过阈值的时候,会自动进行摘要
在缓冲区中,会保存最近的k条对话
比较久的对话会被删除,在删除前会进行摘要

In [26]:
# 首先依旧导入我们的模块
from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

from langchain.memory import ConversationSummaryBufferMemory
from langchain.llms import Tongyi

llm = Tongyi(
    dashscope_api_key = api_key,
    model = "Qwen-max",
    temperature = 0
)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit= 10,
    return_messages = True
)

memory.save_context(
    {
        "input": "你好,帮我找下南秋SouthAki"
    },
    {
        "output": "sorry, who is 南秋SouthAki?"
    }
)

memory.save_context(
    {
        "input": "南秋SouthAki是一个前端开发工程师"
    },
    {
        "output": "ok,i know"
    }
)

memory.save_context(
    {
        "input": "今天他要讲一门关于华为仓颉编程的课程"
    },
    {
        "output": "ok,i know.do you need more information?"
    }
)

memory.load_memory_variables({})

c:\Users\xiele\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'history': [SystemMessage(content="The human is searching for 南秋SouthAki, a front-end development engineer who is scheduled to give a lecture today on Huawei's仓颉programming language. The AI has become aware of this and asks if the human needs further information.")]}

ok ,下面这个是token计算的内容
使用到了`Conversation Token Buffer`
运用token,来决定什么时候刷新内存

In [27]:
from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import Tongyi

llm = Tongyi(
    model = "Qwen-max",
    dashscope_api_key = api_key,
    temperature = 0
)

memory = ConversationTokenBufferMemory(
    llm = llm,
    max_token_limit= 150
)

memory.save_context(
    {
        "input": "你好,帮我找下南秋SouthAki"
    },
    {
        "output": "sorry, who is 南秋SouthAki?"
    }
)

memory.save_context(
    {
        "input": "南秋SouthAki是一个前端开发工程师"
    },
    {
        "output": "ok,i know"
    }
)

memory.save_context(
    {
        "input": "今天他要讲一门关于华为仓颉编程的课程"
    },
    {
        "output": "ok,i know.do you need more information?"
    }
)

memory.save_context(
    {
        "input": "不需要资料了,谢谢"
    },
    {
        "output": "All right, see you next time."
    }
)

memory.load_memory_variables({})

{'history': 'AI: sorry, who is 南秋SouthAki?\nHuman: 南秋SouthAki是一个前端开发工程师\nAI: ok,i know\nHuman: 今天他要讲一门关于华为仓颉编程的课程\nAI: ok,i know.do you need more information?\nHuman: 不需要资料了,谢谢\nAI: All right, see you next time.'}

以上的都是短时记忆的实现
但是,当我们关机等的时候,我们有些数据不限丢失,于是我们需要实现长时记忆,来保存我们的数据
langchain是使用向量数据库来存储之前的对话内容,有的向量数据库服务还提供自动摘要,每次对话的时候,都会从向量数据库里查询最相关的文档或者历史对话

In [31]:
# 首先导入我们的模块
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS

memory = ConversationBufferMemory()
memory.save_context(
    {
        "input": "你好,帮我找下南秋SouthAki"
    },
    {
        "output": "sorry, who is 南秋SouthAki?"
    }
)

memory.save_context(
    {
        "input": "南秋SouthAki是一个前端开发工程师"
    },
    {
        "output": "ok,i know"
    }
)

memory.save_context(
    {
        "input": "今天他要讲一门关于华为仓颉编程的课程"
    },
    {
        "output": "ok,i know.do you need more information?"
    }
)

memory.save_context(
    {
        "input": "不需要资料了,谢谢"
    },
    {
        "output": "All right, see you next time."
    }
)

vectorstore = FAISS.from_texts(
    memory.buffer.split("\n"),
    HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")
)

FAISS.save_local(vectorstore, "test_faiss")

FAISS.load_local("test_faiss", HuggingFaceEmbeddings(model_name="BAAI/bge-small-en"),allow_dangerous_deserialization=True).similarity_search("SouthAki做什么职业?")

[Document(page_content='Human: 你好,帮我找下南秋SouthAki'),
 Document(page_content='Human: 南秋SouthAki是一个前端开发工程师'),
 Document(page_content='AI: sorry, who is 南秋SouthAki?'),
 Document(page_content='Human: 不需要资料了,谢谢')]

In [32]:
# 我们来测试一下是否有成功写入向量数据库
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import VectorStoreRetrieverMemory

r1 = FAISS.load_local("test_faiss", embeddings=HuggingFaceEmbeddings(model_name="BAAI/bge-small-en"),allow_dangerous_deserialization=True)
r2 = r1.as_retriever(
    search_kwargs={"k": 1}
)

memory2 = VectorStoreRetrieverMemory(retriever=r2)

memory2.load_memory_variables({"prompt": "SouthAki是什么职业"})

{'history': 'Human: 南秋SouthAki是一个前端开发工程师'}

在链上使用memory
- LLMChain
- ConversationChain
- 自定义
- 同一个链合并使用多个记忆
- 给多参数链增加记忆

第一个我们来实现的就是关于LLMChain的memory

In [11]:
# 首先依旧导入模块
from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

from langchain.llms import Tongyi
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 自定义一个模版
template = """
    你是一个可以和人类对话的AI机器人
    {chat_history}
    人类:{human_input}
    机器人:
"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"],
    template=template,
)
memory = ConversationBufferMemory(
    memory_key= "chat_history"
)

llm = Tongyi(
    dashscope_api_key = api_key,
    temperature = 0,
    model = "Qwen-max"
)

chain = LLMChain(
    llm = llm,
    memory = memory,
    prompt = prompt,
    verbose = True
)

chain.predict(human_input= '你好')
chain.predict(human_input= '我叫SouthAki,是一个前端工程师,你呢')
chain.predict(human_input= '我喜欢玩游戏,最喜欢玩Minecraft,你知道这个游戏吗?')
chain.predict(human_input= '你还记得我叫什么名字吗?')



> Entering new LLMChain chain...
Prompt after formatting:

    你是一个可以和人类对话的AI机器人
    
    人类:你好
    机器人:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    你是一个可以和人类对话的AI机器人
    Human: 你好
AI: 你好！有什么问题或者需要帮助的吗？
    人类:我叫SouthAki,是一个前端工程师,你呢
    机器人:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    你是一个可以和人类对话的AI机器人
    Human: 你好
AI: 你好！有什么问题或者需要帮助的吗？
Human: 我叫SouthAki,是一个前端工程师,你呢
AI: 你好，SouthAki！我是AI助手，专门设计来帮助解答问题和提供信息。我在乎你的前端开发相关的疑问，或者其他任何话题，我们都可以聊聊。有什么我可以帮助你的吗？
    人类:我喜欢玩游戏,最喜欢玩Minecraft,你知道这个游戏吗?
    机器人:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    你是一个可以和人类对话的AI机器人
    Human: 你好
AI: 你好！有什么问题或者需要帮助的吗？
Human: 我叫SouthAki,是一个前端工程师,你呢
AI: 你好，SouthAki！我是AI助手，专门设计来帮助解答问题和提供信息。我在乎你的前端开发相关的疑问，或者其他任何话题，我们都可以聊聊。有什么我可以帮助你的吗？
Human: 我喜欢玩游戏,最喜欢玩Minecraft,你知道这个游戏吗?
AI: 当然，Minecraft是一款非常受欢迎的沙盒游戏，由 Mojang Studios 开发。在游戏中，玩家可以在三维空间中建造、探索和生存，使用各种不同的方块和资源。你可以创造自己的建筑、红石电路、甚至是完全自定义的游戏模式。Minec

'当然，你是SouthAki，一个喜欢玩Minecraft的前端工程师。有什么关于Minecraft或者其他话题想要讨论的吗？'

接下来的是对话的chatModel

In [18]:
# 首先依旧是导入模块
from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

from langchain_community.chat_models import ChatTongyi
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.chains import LLMChain
from langchain.schema import SystemMessage

# 定义提问提示模版
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content = "你好,我是一个可以和人类进行自然语言对话的AI机器人",
            role = "system"
        ),
        # Meassagesplaceholder: 是一个占位符,这里的作用是传入我们的历史聊天信息
        MessagesPlaceholder(
            variable_name= "chat_history"
        ),
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        )
    ]
)

# 先来看看格式是否符合我们预期
# print(prompt.format_prompt(human_input="你好",chat_history = []))

memory = ConversationBufferMemory(
    memory_key= "chat_history",
    return_messages = True
)

llm = ChatTongyi(
    dashscope_api_key = api_key,
    temperature = 0,
    model_name = "qwen-vl-max"
)

chain = LLMChain(
    llm = llm,
    memory = memory,
    prompt = prompt,
    verbose = True
)

# chain.predict(human_input= '你好')
chain.predict(human_input= '我叫SouthAki,是一个前端工程师,你呢')
chain.predict(human_input= '我喜欢玩游戏,最喜欢玩Minecraft,你知道这个游戏吗?')
chain.predict(human_input= '你还记得我叫什么名字吗?')



> Entering new LLMChain chain...
Prompt after formatting:
System: 你好,我是一个可以和人类进行自然语言对话的AI机器人
Human: 我叫SouthAki,是一个前端工程师,你呢

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: 你好,我是一个可以和人类进行自然语言对话的AI机器人
Human: 我叫SouthAki,是一个前端工程师,你呢
AI: 你好，我是来自阿里云的语言模型，我叫通义千问。作为一个AI助手，我的目标是帮助用户获得准确、有用的信息，解决他们的问题和困惑。我会不断学习和进步，不断提升自己的能力，为用户提供更好的服务。如果您有任何问题或需要帮助，请随时告诉我，我会尽力提供支持。
Human: 我喜欢玩游戏,最喜欢玩Minecraft,你知道这个游戏吗?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: 你好,我是一个可以和人类进行自然语言对话的AI机器人
Human: 我叫SouthAki,是一个前端工程师,你呢
AI: 你好，我是来自阿里云的语言模型，我叫通义千问。作为一个AI助手，我的目标是帮助用户获得准确、有用的信息，解决他们的问题和困惑。我会不断学习和进步，不断提升自己的能力，为用户提供更好的服务。如果您有任何问题或需要帮助，请随时告诉我，我会尽力提供支持。
Human: 我喜欢玩游戏,最喜欢玩Minecraft,你知道这个游戏吗?
AI: Minecraft是一款非常受欢迎的沙盒式建造游戏，由瑞典开发团队Mojang Studios开发。在游戏中，玩家可以在一个三维空间中自由探索、采集资源，并使用这些资源建造建筑物、工具和其他各种物品。游戏提供了生存模式和创造模式等多种玩法，玩家可以根据自己的喜好选择不同的模式来体验游戏的乐趣。Minecraft以其极高的自由度和丰富的可玩性受到了全球众多玩家的喜爱。
Human: 你还记得我叫什么名字吗?

> Finished chain.


'你是叫做 SouthAki 的前端工程师，很高兴认识你！'

下一个的是对话链
ConversationChain

In [22]:
# 我们来尝试一下,使用预制链提供的东西
# 首先导入模块
from langchain.chains import ConversationChain
from langchain.llms import Tongyi
from langchain.memory import ConversationBufferMemory

from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

# 依旧是定义我们的大模型
llm = Tongyi(
    model = "Qwen",
    dashscope_api_key = api_key,
    temperature = 0
)
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key= "history"
)
chain = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)

chain.predict(input="你好")
chain.predict(input="我叫SouthAki,你呢")
chain.predict(input="今天天气真好,我感觉我想去北京旅游")
chain.predict(input="帮我做个一日游攻略")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: 你好
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='你好'), AIMessage(content='你好！有什么问题或者需要帮助的吗？我可以提供各种信息，比如天气预报、新闻更新、食谱建议、电影推荐，甚至是帮你解决数学问题。只要你告诉我你想要什么，我都会尽力协助你。')]
Human: 我叫SouthAki,你呢
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The

'当然可以，SouthAki。以下是一份北京一日游的攻略，旨在让你在短时间内尽可能多地体验北京的魅力：\n\n**早晨：**\n1. **天安门广场**（6:30-8:00）：早起参观天安门广场，观看升旗仪式（具体时间可能因日期而异，建议提前查询），然后游览人民英雄纪念碑、毛主席纪念堂。\n\n2. **故宫博物院**（8:30-12:00）：从天安门广场步行即可到达故宫，这里是中国明清两代皇家宫殿，有很多历史文物和故事。\n\n**午餐：**\n3. **王府井小吃街**（12:00-13:00）：品尝北京地道小吃，如炸酱面、艾窝窝、糖葫芦等。\n\n**下午：**\n4. **景山公园**（13:30-14:30）：从故宫北门出来，步行到景山公园，登高望远，可以看到故宫全景。\n\n5. **北海公园**（15:00-17:00）：接着步行或乘坐公交车到北海公园，划船、欣赏白塔，感受古典园林之美。\n\n**傍晚：**\n6. **南锣鼓巷**（17:30-19:00）：逛一逛这条充满老北京风情的胡同，体验当地文化，还可以购买一些手工艺品或纪念品。\n\n**晚餐：**\n7. **簋街**（19:30-21:00）：前往簋街享用晚餐，这里有很多著名的餐馆，可以尝试正宗的北京烤鸭。\n\n**夜生活：**\n8. **后海酒吧街**（21:00-23:00）：结束晚餐后，可以到后海散步，享受北京的夜生活，这里有各种风格的酒吧和音乐演出。\n\n请注意，这是一条紧凑的行程，可能需要根据实际情况进行调整。如果时间允许，你还可以考虑在一天中的某个时段参观附近的恭王府或国家博物馆。希望这个攻略对你有所帮助，祝你在北京度过愉快的一天！'

可以看到使用预制链确实很轻松的实现了
但是某些时候,我们需要通过定制,使得llm输出更符合我们的预期

In [28]:
# 自定义一下,对其进行覆盖
# 首先依旧导入我们的模块
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.llms import Tongyi

from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

# 定义大模型
llm = Tongyi(
    dashscope_api_key = api_key,
    temperature = 0,
    model = "Qwen-max"
)

# 设置问题模版
template = """
    下面是一段AI与人类的对话,AI会针对人类的问题,提供尽可能详细回答,如果AI不知道答案,会直接回复'Soory, I don't know the answer.'
    当前对话:{history}
    human:{input}
    AI助手:
"""

prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=template
)

chain = ConversationChain(
    llm = llm,
    memory = ConversationBufferMemory(
        # 这里为什么要有一个ai_prefix呢?
        # 因为我们上面修改了那个问题的信息回复人,如果不用占位符,就会导致报错
        ai_prefix = "AI助手",
        return_messages = True
    ),
    prompt = prompt,
    verbose = True
)

# 测试 提出问题
chain.predict(input = "你好")
chain.predict(input = "今天天气真好,适合旅游啊")
chain.predict(input = "我最近想去南京,你有什么推荐的吗,帮我做个一日游攻略出来")
chain.predict(input = "我叫什么名字?")



> Entering new ConversationChain chain...
Prompt after formatting:

    下面是一段AI与人类的对话,AI会针对人类的问题,提供尽可能详细回答,如果AI不知道答案,会直接回复'Soory, I don't know the answer.'
    当前对话:[]
    human:你好
    AI助手:


> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:

    下面是一段AI与人类的对话,AI会针对人类的问题,提供尽可能详细回答,如果AI不知道答案,会直接回复'Soory, I don't know the answer.'
    当前对话:[HumanMessage(content='你好'), AIMessage(content='你好！有什么问题或者需要帮助的吗？')]
    human:今天天气真好,适合旅游啊
    AI助手:


> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:

    下面是一段AI与人类的对话,AI会针对人类的问题,提供尽可能详细回答,如果AI不知道答案,会直接回复'Soory, I don't know the answer.'
    当前对话:[HumanMessage(content='你好'), AIMessage(content='你好！有什么问题或者需要帮助的吗？'), HumanMessage(content='今天天气真好,适合旅游啊'), AIMessage(content='是的，天气好的时候确实很适合出游。如果你正在计划一次旅行，我可以帮你提供一些旅行建议，比如目的地推荐、行程规划或者天气预报等信息。你有特定的目的地或者需求吗？')]
    human:我最近想去南京,你有什么推荐的吗,帮我做个一日游攻略出来
    AI助手:


> Finished chain.


> Entering new ConversationChain chain...
Prompt af

'你的名字我没有记录，因为你之前没有告诉我。如果你愿意告诉我，我很乐意记住它。'

下面,我们展示一下同一条链合并使用多个memory

In [35]:
# 首先还是导入模块
from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

from langchain.llms import Tongyi
from langchain.chains import ConversationChain
from langchain.memory import (
    ConversationBufferMemory,
    ConversationSummaryMemory,
    CombinedMemory
)
from langchain.prompts import PromptTemplate

# 定义大模型
llm = Tongyi(
    dashscope_api_key = api_key,
    model = "Qwen-max",
    temperature = 0
)

# 使用ConversationSummaryMemory对对话进行总结
summary = ConversationSummaryMemory(
    llm= llm,
    input_key= 'input'
)

# 使用ConversationBufferMemory对对话进行缓存
cov_memory = ConversationBufferMemory(
    memory_key= 'history_now',
    input_key= 'input'
)

memory = CombinedMemory(
    memories=[summary, cov_memory]
)

template = """
    下面是一段AI与人类的对话,AI会针对人类的问题,尽力的给出详细答案,如果不知道问题的答案,会回答'Sorry, I don't know the answer to that question.'
    之前的对话摘要:{history}
    当前对话:{history_now}
    Human:{input}
    AI:
"""

prompt = PromptTemplate(
    template= template,
    input_variables=['history', 'history_now', 'input']
)

chain = ConversationChain(
    llm = llm,
    memory = memory,
    prompt = prompt,
    verbose = True
)

chain.run("你好")
chain.run("我想知道加密货币的相关知识")
chain.run("好厉害啊,那你对BTC有认识吗？")
chain.run("那ETH呢?")



> Entering new ConversationChain chain...
Prompt after formatting:

    下面是一段AI与人类的对话,AI会针对人类的问题,尽力的给出详细答案,如果不知道问题的答案,会回答'Sorry, I don't know the answer to that question.'
    之前的对话摘要:
    当前对话:
    Human:你好
    AI:


> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:

    下面是一段AI与人类的对话,AI会针对人类的问题,尽力的给出详细答案,如果不知道问题的答案,会回答'Sorry, I don't know the answer to that question.'
    之前的对话摘要:The human greets the AI in Chinese, and the AI responds with a friendly greeting, offering assistance.
    当前对话:Human: 你好
AI: 你好！有什么问题我可以帮助你解答吗？
    Human:我想知道加密货币的相关知识
    AI:


> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:

    下面是一段AI与人类的对话,AI会针对人类的问题,尽力的给出详细答案,如果不知道问题的答案,会回答'Sorry, I don't know the answer to that question.'
    之前的对话摘要:The human inquires about cryptocurrency, and the AI explains that it is a digital or virtual currency that uses encryption techniques for secure transactions and asset transfers, operating inde

'ETH是以太坊（Ethereum）的缩写，是以太坊网络的原生加密货币。以太坊是由 Vitalik Buterin 在2013年提出的，并在2015年正式上线。与比特币主要作为价值存储不同，以太坊是一个开放的、分散的平台，允许开发人员构建和运行去中心化的应用程序（DApps）以及智能合约。\n\n以太坊的主要特点包括：\n\n1. **智能合约**：以太坊网络支持智能合约，这是一种自动执行的程序，能够在满足预定条件时执行预设的操作，这为创建去中心化的金融产品、游戏、身份验证系统等提供了可能。\n2. **gas和交易费**：在以太坊上执行操作需要消耗“gas”，这是以太币的小额支付，用于支付执行智能合约所需的计算资源。\n3. **升级计划**：以太坊正在经历从工作量证明（PoW）到权益证明（Proof-of-Stake, PoS）的过渡，这个过程被称为“以太坊2.0”。PoS将减少能源消耗，提高效率，并增加网络的安全性。\n4. **可扩展性挑战**：尽管以太坊是目前最广泛使用的智能合约平台，但它面临着交易速度慢和交易费用高的问题。社区正在努力通过分片和其他技术解决方案来解决这些问题。\n\n如果你对以太坊的生态系统、智能合约的工作原理、以太坊2.0的进展，或者其他相关话题感兴趣，欢迎继续提问。'

下面来讲一个多参数链增加记忆的实现

In [37]:
# 首先导入模块
from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter

with open('./letter.txt') as f:
    text = f.read()

    # 对文本进行切分
    text_splitter = CharacterTextSplitter(chunk_size=1000, separator="\n")
    texts = text_splitter.split_text(text)

    # 使用HuggingFaceEmbeddings进行向量化
    embeedings = HuggingFaceEmbeddings(
        model_name="BAAI/bge-small-en"
    )
    # 使用chroma去存储向量
    docsearch = Chroma.from_texts(
        texts,
        embeedings
    )
    query = "这家公司有什么新策略?"
    docs = docsearch.similarity_search(query = query)

当上面成功后,我们需要构建一个问答对话链

In [41]:
# 首先导入模块
from dotenv import find_dotenv, load_dotenv
import os
# 加载 API key
load_dotenv(find_dotenv())
api_key = os.getenv("DASHSCOPE_API_KEY")

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter

with open('./letter.txt') as f:
    text = f.read()

    # 对文本进行切分
    text_splitter = CharacterTextSplitter(chunk_size=1000, separator="\n")
    texts = text_splitter.split_text(text)

    # 使用HuggingFaceEmbeddings进行向量化
    embeedings = HuggingFaceEmbeddings(
        model_name="BAAI/bge-small-en"
    )
    # 使用chroma去存储向量
    docsearch = Chroma.from_texts(
        texts,
        embeedings
    )
    query = "这家公司有什么新策略?"
    docs = docsearch.similarity_search(query = query)

# 首先依旧是导入模块
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import Tongyi
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

# 定义大模型
llm = Tongyi(
    model = "Qwen-max",
    dashscope_api_key = api_key,
    temperature = 0
)

template = """
    下面是一段AI与人类的对话,AI会针对人类的问题,提供尽可能详细的回答,如果AI不知道这个问题的答案,会返回一段字符串'Sorry, I don't know the answer.',依旧参考一下相关文档以及历史对话信息,AI会据此组织最终回答的内容.{context}
    {chat_history}
    Human: {human_input}
    AI:
"""

prompt = PromptTemplate(
    template= template,
    input_variables = ["context","chat_history","human_input"]
)

# 使用ConversationBufferMemory对对话进行缓存
memory = ConversationBufferMemory(
    memory_key= "chat_history",
    input_key= "human_input",
    return_messages= True
)

# 加载我们的对话链
chain = load_qa_chain(
    llm= llm,
    memory= memory,
    prompt= prompt,
    verbose= True,
    chain_type= "stuff"
)

# 调用一下上面的文档和我们的问答历史
chain({"input_documents":docs,"human_input":"公司有什么新策略?"})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    下面是一段AI与人类的对话,AI会针对人类的问题,提供尽可能详细的回答,如果AI不知道这个问题的答案,会返回一段字符串'Sorry, I don't know the answer.',依旧参考一下相关文档以及历史对话信息,AI会据此组织最终回答的内容.[Generated with ChatGPT]
Confidential Document - For Internal Use Only
Date: July 1, 2023
Subject: Updates and Discussions on Various Topics
Dear Team,
I hope this email finds you well. In this document, I would like to provide you with some important updates and discuss various topics that require our attention. Please treat the information contained herein as highly confidential.
Security and Privacy Measures
As part of our ongoing commitment to ensure the security and privacy of our customers' data, we have implemented robust measures across all our systems. We would like to commend John Doe (email: john.doe@example.com) from the IT department for his diligent work in enhancing our network security. Moving forward, we kindly remind everyone to stric

{'input_documents': [Document(page_content="[Generated with ChatGPT]\nConfidential Document - For Internal Use Only\nDate: July 1, 2023\nSubject: Updates and Discussions on Various Topics\nDear Team,\nI hope this email finds you well. In this document, I would like to provide you with some important updates and discuss various topics that require our attention. Please treat the information contained herein as highly confidential.\nSecurity and Privacy Measures\nAs part of our ongoing commitment to ensure the security and privacy of our customers' data, we have implemented robust measures across all our systems. We would like to commend John Doe (email: john.doe@example.com) from the IT department for his diligent work in enhancing our network security. Moving forward, we kindly remind everyone to strictly adhere to our data protection policies and guidelines. Additionally, if you come across any potential security risks or incidents, please report them immediately to our dedicated team